In [1]:
# Dependencies and Setup
import csv 
from config import api_key
import pandas as pd
import numpy as np
import requests
import time
import json
import pymongo
from sqlalchemy import create_engine
import datetime as dt
url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx"
units = "metric"

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
# csv_file = 'Resources/2018_airline_delay_causes_csv'
# airline_data_df = pd.read_csv(csv_file, delimiter = ',')
# airline_data_df.head()

In [4]:
def month_json_to_df(city_name):
    start_date=["2018-01-01","2018-02-01","2018-03-01","2018-04-01","2018-05-01","2018-06-01","2018-07-01","2018-08-01","2018-09-01","2018-10-01","2018-11-01","2018-12-01"]
    end_date=["2018-01-31","2018-02-28","2018-03-31","2018-04-30","2018-05-31","2018-06-30","2018-07-31","2018-08-31","2018-09-30","2018-10-31","2018-11-30","2018-12-31"]
    for i in range(12):
        query_url = f"{url}?key={api_key}&q={city_name}&format=json&extra=2018-01-01&date={start_date[i]}&enddate={end_date[i]}"
        response=requests.get(query_url)
        json_file = response.json()
        city = json_file['data']['request'][0]['query']
    
        date=[]
        weather_desc=[]
        for weather_row in json_file['data']['weather']:
            date.append(weather_row['date'])
            weather_desc.append(weather_row['hourly'][0]['weatherDesc'][0]['value'])
        if i ==0:
            final_DF=pd.DataFrame({'city':city, 'date':date, 'weather':weather_desc})
        else:
            temp_df=pd.DataFrame({'city':city, 'date':date, 'weather':weather_desc})
            final_DF=pd.concat([final_DF, temp_df])
    return final_DF

In [5]:
lax_df=month_json_to_df('Los Angeles')
lax_df.to_csv('Resources/lax_df.csv')
#lax_df = lax_df.rename(columns= {"city":"city_name", "date":"date_of","weather":"weather_desc"})
lax_df.head()

,city,date,weather
0,"Los Angeles, United States of America",2018-01-01,Clear
1,"Los Angeles, United States of America",2018-01-02,Clear
2,"Los Angeles, United States of America",2018-01-03,Clear
3,"Los Angeles, United States of America",2018-01-04,Clear
4,"Los Angeles, United States of America",2018-01-05,Clear


In [6]:
chi_df=month_json_to_df("Chicago")
chi_df.head()

,city,date,weather
0,"Chicago, United States of America",2018-01-01,Clear
1,"Chicago, United States of America",2018-01-02,Clear
2,"Chicago, United States of America",2018-01-03,Clear
3,"Chicago, United States of America",2018-01-04,Partly cloudy
4,"Chicago, United States of America",2018-01-05,Clear


In [7]:
dfw_df=month_json_to_df("Dallas")
dfw_df.head()

,city,date,weather
0,"Dallas, United States of America",2018-01-01,Partly cloudy
1,"Dallas, United States of America",2018-01-02,Clear
2,"Dallas, United States of America",2018-01-03,Clear
3,"Dallas, United States of America",2018-01-04,Clear
4,"Dallas, United States of America",2018-01-05,Overcast


In [8]:
atl_df=month_json_to_df("Atlanta")
atl_df.head()

,city,date,weather
0,"Atlanta, United States of America",2018-01-01,Partly cloudy
1,"Atlanta, United States of America",2018-01-02,Clear
2,"Atlanta, United States of America",2018-01-03,Overcast
3,"Atlanta, United States of America",2018-01-04,Clear
4,"Atlanta, United States of America",2018-01-05,Clear


In [9]:
den_df=month_json_to_df("Denver")
den_df.head()

,city,date,weather
0,"Denver, United States of America",2018-01-01,Clear
1,"Denver, United States of America",2018-01-02,Clear
2,"Denver, United States of America",2018-01-03,Clear
3,"Denver, United States of America",2018-01-04,Clear
4,"Denver, United States of America",2018-01-05,Clear


In [10]:
#den_df.columns

In [11]:
final = pd.concat([lax_df, dfw_df, chi_df, atl_df, den_df])
final_weather = final


In [12]:
final_weather.weather.unique()

array(['Clear', 'Cloudy', 'Partly cloudy', 'Moderate rain',
       'Moderate rain at times', 'Light drizzle', 'Overcast',
       'Light rain', 'Light rain shower', 'Moderate or heavy rain shower',
       'Patchy rain possible', 'Patchy light drizzle', 'Mist',
       'Thundery outbreaks possible', 'Fog', 'Freezing fog'], dtype=object)

In [13]:
final['weather_val']=final['weather'].replace({'Clear':15, 'Overcast':14,'Partly cloudy':13, 'Cloudy':12, 'Fog':11,\
                          'Freezing fog':10, 'Mist':9, 'Patchy light drizzle':8, 'Light drizzle':7, 'Patchy rain possible':6,\
                         'Light rain':5, 'Light rain shower':4, 'Moderate rain at times':3,'Moderate rain':2, 'Moderate or heavy rain shower':1,\
                         'Thundery outbreaks possible':0})


In [14]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1825 entries, 0 to 30
Data columns (total 4 columns):
city           1825 non-null object
date           1825 non-null object
weather        1825 non-null object
weather_val    1825 non-null int64
dtypes: int64(1), object(3)
memory usage: 71.3+ KB


In [15]:
final['date']=pd.to_datetime(final['date'])

In [16]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1825 entries, 0 to 30
Data columns (total 4 columns):
city           1825 non-null object
date           1825 non-null datetime64[ns]
weather        1825 non-null object
weather_val    1825 non-null int64
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 71.3+ KB


In [17]:
final['month']= final['date'].dt.month

In [18]:
final['year']= final['date'].dt.year

In [19]:
final.head()

,city,date,weather,weather_val,month,year
0,"Los Angeles, United States of America",2018-01-01,Clear,15,1,2018
1,"Los Angeles, United States of America",2018-01-02,Clear,15,1,2018
2,"Los Angeles, United States of America",2018-01-03,Clear,15,1,2018
3,"Los Angeles, United States of America",2018-01-04,Clear,15,1,2018
4,"Los Angeles, United States of America",2018-01-05,Clear,15,1,2018


In [20]:
final.columns

Index(['city', 'date', 'weather', 'weather_val', 'month', 'year'], dtype='object')

In [21]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1825 entries, 0 to 30
Data columns (total 6 columns):
city           1825 non-null object
date           1825 non-null datetime64[ns]
weather        1825 non-null object
weather_val    1825 non-null int64
month          1825 non-null int64
year           1825 non-null int64
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 99.8+ KB


In [22]:
final[['city','country']] = final.city.str.split(expand= True,pat=',')

In [23]:
final.head()

,city,date,weather,weather_val,month,year,country
0,Los Angeles,2018-01-01,Clear,15,1,2018,United States of America
1,Los Angeles,2018-01-02,Clear,15,1,2018,United States of America
2,Los Angeles,2018-01-03,Clear,15,1,2018,United States of America
3,Los Angeles,2018-01-04,Clear,15,1,2018,United States of America
4,Los Angeles,2018-01-05,Clear,15,1,2018,United States of America


In [53]:
final_df = final.groupby(['year','month','city'],as_index=False).mean()#.agg(lambda x:x.value_counts().index[0])

final_df.weather_val = np.round(final_df.weather_val)

In [60]:
final_df

,year,month,city,index,weather_val
0,2018,1,Atlanta,15.0,14.0
1,2018,1,Chicago,15.0,14.0
2,2018,1,Dallas,15.0,14.0
3,2018,1,Denver,15.0,15.0
4,2018,1,Los Angeles,15.0,14.0
5,2018,2,Atlanta,13.5,13.0
6,2018,2,Chicago,13.5,13.0
7,2018,2,Dallas,13.5,13.0
8,2018,2,Denver,13.5,14.0
9,2018,2,Los Angeles,13.5,14.0


In [61]:
final_df[final_df.city=='Los Angeles']

,year,month,city,index,weather_val
4,2018,1,Los Angeles,15.0,14.0
9,2018,2,Los Angeles,13.5,14.0
14,2018,3,Los Angeles,15.0,12.0
19,2018,4,Los Angeles,14.5,15.0
24,2018,5,Los Angeles,15.0,14.0
29,2018,6,Los Angeles,14.5,15.0
34,2018,7,Los Angeles,15.0,15.0
39,2018,8,Los Angeles,15.0,15.0
44,2018,9,Los Angeles,14.5,15.0
49,2018,10,Los Angeles,15.0,14.0


In [54]:
final_df.weather_val.unique()

array([14., 15., 13., 12.])

In [55]:
final_df.weather_val.value_counts()

14.0    40
15.0    13
13.0     6
12.0     1
Name: weather_val, dtype: int64

In [46]:
city_list = final.city.unique()

In [59]:
for city in city_list:
    print(city)
#     print(final[(final.city==city)&(final.month == 5)]['weather'].value_counts())
    print(final[(final.city==city)]['weather'].value_counts())

    print('-----------------------')

Los Angeles
Clear                            291
Partly cloudy                     41
Cloudy                            16
Light rain                         4
Patchy rain possible               3
Overcast                           3
Light rain shower                  2
Patchy light drizzle               1
Moderate or heavy rain shower      1
Moderate rain                      1
Light drizzle                      1
Moderate rain at times             1
Name: weather, dtype: int64
-----------------------
Dallas
Clear                          215
Partly cloudy                   64
Overcast                        47
Cloudy                          27
Thundery outbreaks possible      9
Mist                             3
Name: weather, dtype: int64
-----------------------
Chicago
Clear                          171
Overcast                        84
Partly cloudy                   65
Cloudy                          28
Mist                             6
Thundery outbreaks possible      6
Fog  